In [16]:
import dawdreamer as daw
from scipy.io import wavfile
SAMPLE_RATE = 44100
engine = daw.RenderEngine(SAMPLE_RATE, 512)  # 512 block size #オーディオ処理のエンジンをインスタンス化
faust_processor = engine.make_faust_processor("faust") #FAUST Processorのインスタンス化
faust_processor.set_dsp_string(
    """
    declare name "MySine";
    freq = hslider("freq", 440, 0, 20000, 0);
    gain = hslider("vol[unit:dB]", 0, -120, 20, 0) : ba.db2linear;
    process = freq : os.osc : _*gain <: si.bus(2);
    """
    )
faust_processor.get_parameters_description()
engine.load_graph([
                   (faust_processor, [])
])
faust_processor.set_parameter("/MySine/freq", 880.)  # 440 Hz
faust_processor.set_parameter("/MySine/vol", -6.)  # -6 dB volume

engine.set_bpm(120.)
engine.render(1., beats=True)  # render 4 beats.(duration setting)

audio = engine.get_audio()  # shaped (2, N samples) #stereo output
wavfile.write('sine_demo.wav', SAMPLE_RATE, audio.transpose())
print(faust_processor.get_parameters_description())


# Change settings and re-render
faust_processor.set_parameter("/MySine/freq", 880.)  # 880 Hz
engine.render(4., beats=True)
# and so on...

[{'index': 0, 'name': '/MySine/freq', 'numSteps': 9223372036854775807, 'isDiscrete': False, 'label': 'freq', 'min': 0.0, 'max': 20000.0, 'step': 0.0, 'value': 880.0}, {'index': 1, 'name': '/MySine/vol', 'numSteps': 9223372036854775807, 'isDiscrete': False, 'label': 'vol', 'min': -120.0, 'max': 20.0, 'step': 0.0, 'value': -6.0}]


True

In [3]:
!ls

Dawdreamer_Test.ipynb README.md             Vital.vst
LIRA-8.component      Synth1.component      Vital.vst3
LIRA-8.vst            VCV Rack 2 FX.vst     neutone.vst3
LIRA-8.vst3           VCV Rack 2.vst        sine_demo.wav


In [8]:
# https://twitter.com/DoItRealTime/status/1294394705040609282

import dawdreamer as daw #dawdreamerをインポート
import numpy as np #numpyをインポート
import librosa #librosaをインポート
from scipy.io import wavfile


SAMPLE_RATE, BLOCK_SIZE = 44100, 512
engine = daw.RenderEngine(SAMPLE_RATE, BLOCK_SIZE)

my_synth_pass = "/Library/Audio/Plug-Ins/VST3/Dexed.vst3"
# OSXでサポートされてるのは以下のissueに記載あり.自分が持っている複数のプラグインは読み込めなかった.
# https://github.com/DBraun/DawDreamer/issues/120
# https://github.com/DBraun/DawDreamer/wiki/Plugin-Compatibility

my_preset_path = ""
my_midi_path = "/Users/tsugumasayutani/Git/Dawdreamer_Test/01Prelude.mid"
reverb_pass = ""
wav_data_path = "/Users/tsugumasayutani/Git/Dawdreamer_Test/sine_demo.wav"

# Make a processor with a unique name "my_synth", which we must remember later.
synth = engine.make_plugin_processor("my_synth", my_synth_pass)
#synth.load_preset("my_preset_path")
#print(synth.get_parameters_description())
synth.load_midi(my_midi_path)
# We can also add notes one at a time.
#synth.add_note(67, 127, 0.5, .25)  # (MIDI note, velocity, start sec, duration sec)

vocals_data = librosa.load(wav_data_path, duration=10.) # use librosa for audio load

# A graph is meaningfully ordered list of tuples.
# In each tuple, the first item is an audio processor
# The second item is this audio processor's list of unique names of input processors.
# Audio from the last tuple's processor will be retrieved by engine.get_audio()

graph = [
    (synth, []),
    (engine.make_reverb_processor("basic_reverb"), ["my_synth"]),
#    (engine.make_plugin_processor("more_reverb", "reverb_pass"), ["basic_reverb"]),
#    (engine.make_playback_processor("my_vocals", vocals_data), []),
#    (engine.make_filter_processor("basic_filter", "high", 1000.0, .7, 1.), ["my_vocal"]),
#    (engine.make_add_processor("basic_add"), ["more_reverb","basic_filter"]),
#    (engine.make_compressor_processor("basic_compressor", 0, 1., 2., 50.), ["basic_add"]),
]

engine.load_graph(graph)
engine.render(10.)  # render 10 seconds audio.
audio = engine.get_audio()  # Returns python list of lists. The shape is (2, NUM_SAMPLES)
wavfile.write('my_song.wav', SAMPLE_RATE, np.array(audio, np.float32).transpose())

# You can modify processors without recreating the graph.
#synth.load_preset("my_preset_path")
engine.render(10.) # Render audio again!

error: attempt to map invalid URI `/Library/Audio/Plug-Ins/VST3/Dexed.vst3'


AttributeError: 'dawdreamer.dawdreamer.PluginProcessor' object has no attribute 'add_note'

In [ ]:
# https://github.com/DBraun/DawDreamer/issues/1

import librosa
import dawdreamer as daw
import os
SAMPLE_RATE = 44100
song_path = "/Users/tsugumasayutani/Git/Dawdreamer_Test/sine_demo.wav"
vst_effect_path = "/Library/Audio/Plug-Ins/VST3/Dexed.vst3"

engine = daw.RenderEngine(SAMPLE_RATE, 512)
def load_audio_file(file_path, duration=None):
    sig, rate = librosa.load(file_path, duration=duration, mono=False, sr=SAMPLE_RATE)
    assert(rate == SAMPLE_RATE)
    return sig

vocals = load_audio_file(song_path, duration=5.)

import random

while True:
    graph = [
        (engine.make_playback_processor("vocals", vocals), []),
        (engine.make_plugin_processor("reverb", vst_effect_path ), ["vocals"])
        ]
    assert(engine.load_graph(graph))
    # engine.clear()
    os.system("clear")
    # del graph
    print(random.random())

In [3]:
# https://github.com/DBraun/DawDreamer/issues/16

import dawdreamer as daw
SAMPLE_RATE = 44100
engine = daw.RenderEngine(SAMPLE_RATE, 512)
synth = engine.make_synth_processor()
# make_synth_processorはない？

adsr1 = synth.make_adsr()  # for controlling volume
adsr2 = synth.make_adsr()  # for controlling filter cutoff

wavetable = synth.make_wavetable()  # default sine wave based on MIDI key.

filter = synth.make_filter(mode='low')  # low-pass filter
filter.freq = 7000.  # baseline cutoff in Hz.

# connect wavetable to filter and then filter to synth output
wavetable.output.connect(filter.input)  # ugly?
filter.output.connect(synth.output)

routing1 = adsr1.output.connect(wavetable.level, mode='unipolar', amount=1.)  # control volume
routing2 = adsr2.output.connect(filter.freq, mode='unipolar', amount=2000.)  # control cutoff

# finally, make graph
graph = [
    (synth, [])  # no inputs necessary to this synth
]

engine.load_graph(graph)
engine.render(10.)

# change parameters or routing and render again
filter.mode = 'bandpass'  # switch to bandpass filter.
routing2.amount = 3000.

engine.render(10)

synth.remove_routing(routing2)  # remove the filter routing

engine.render(10)

AttributeError: 'dawdreamer.dawdreamer.RenderEngine' object has no attribute 'make_synth_processor'

In [5]:
# https://github.com/DBraun/DawDreamer/issues/54

import dawdreamer as daw
import numpy as np
from scipy.io import wavfile
import librosa

SAMPLE_RATE = 44100
BUFFER_SIZE = 512
DURATION = 15

def load_audio_file(file_path, duration=None):
  sig, rate = librosa.load(file_path, duration=duration, mono=False, sr=SAMPLE_RATE)
  assert(rate == SAMPLE_RATE)
  return sig

engine = daw.RenderEngine(SAMPLE_RATE, BUFFER_SIZE)
meu_plugin = engine.make_plugin_processor("plugin", "/Library/Audio/Plug-Ins/VST3/Dexed.vst3")

meu_plugin.set_parameter(0, 45) # override a specific parameter.
print(meu_plugin.get_parameter(0))  # warning, is this actually 45? Probably not. VST parameters are normally between 0 and 1.
print(meu_plugin.get_parameter_name(0))

multi =  load_audio_file("/Users/tsugumasayutani/Git/Dawdreamer_Test/sine_demo.wav")

playback_processor = engine.make_playback_processor("playback", multi)

graph = [
  (playback_processor , []),  # playback takes no inputs
  (meu_plugin, ["playback"])
]

## this also works
#graph = [
#  (playback_processor , []),  # playback takes no inputs
#  (meu_plugin, [playback_processor.get_name()])
#]

engine.load_graph(graph)
engine.render(DURATION)
audio = engine.get_audio()  

wavfile.write('my_song2.wav', SAMPLE_RATE, audio.transpose()) # don't forget to transpose!

45.0
Cutoff


error: attempt to map invalid URI `/Library/Audio/Plug-Ins/VST3/Dexed.vst3'
